# Código para usar el modelo

In [ ]:
!pip install -q pytube
!pip install -q decord
!pip install -q torchviz
!pip install -q transformers
!pip install -q sentencepiece
!pip install -q datasets
!pip3 install -q imageio==2.4.1

     |████████████████████████████████| 56 kB 1.0 MB/s 
     |████████████████████████████████| 13.6 MB 1.2 MB/s 
     |████████████████████████████████| 5.5 MB 5.2 MB/s 
     |████████████████████████████████| 182 kB 37.8 MB/s 
     |████████████████████████████████| 7.6 MB 24.1 MB/s 
     |████████████████████████████████| 1.3 MB 5.6 MB/s 
     |████████████████████████████████| 451 kB 4.9 MB/s 
     |████████████████████████████████| 115 kB 56.5 MB/s 
     |████████████████████████████████| 212 kB 57.4 MB/s 
     |████████████████████████████████| 127 kB 64.2 MB/s 
     |████████████████████████████████| 3.3 MB 5.4 MB/s 


In [ ]:
# Download from tube
from pytube import YouTube 

# Make a secuence of frames
from decord import VideoReader, cpu
import numpy as np
import math

# View images frames
from PIL import Image

# Images model
import torch
from transformers import VisionEncoderDecoderModel, ViTFeatureExtractor, AutoTokenizer

# Sound model
from IPython.display import Audio
import moviepy.editor as mp
import torch
import librosa
import os
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
import torch

# state bar
from tqdm.auto import tqdm

## Text to text
import torch
from transformers import T5ForConditionalGeneration,T5Tokenizer, AutoModelForSeq2SeqLM, AutoTokenizer

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)2220032/45929032 bytes (4.8%)5013504/45929032 bytes (10.9%)7577600/45929032 bytes (16.5%)10379264/45929032 bytes (22.6%)13205504/45929032 bytes (28.8%)15687680/45929032 bytes (34.2%)18063360/45929032 bytes (39.3%)21078016/45929032 bytes (45.9%)23764992/45929032 bytes (51.7%)26656768/45929032 bytes (58.0%)29376512/45929032 bytes (64.0%)31465472/45929032 bytes (68.5%)

In [ ]:
# Video model
model_image = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
feature_extractor = ViTFeatureExtractor.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
tokenizer_img = AutoTokenizer.from_pretrained("nlpconnect/vit-gpt2-image-captioning")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_image.to(device)

max_length = 16
num_beams = 4
gen_kwargs = {"max_length": max_length, "num_beams": num_beams}

# Sound model
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model_sound = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")     


## Text-text
model = T5ForConditionalGeneration.from_pretrained("Michau/t5-base-en-generate-headline")
tokenizer_txt = T5Tokenizer.from_pretrained("Michau/t5-base-en-generate-headline")

Downloading:   0%|          | 0.00/4.61k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/982M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/228 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/241 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

In [ ]:
# Functions

def sample_frame_indices(clip_len, frame_sample_rate, seg_len):
    converted_len = int(clip_len * frame_sample_rate)
    start_idx = math.floor(seg_len/clip_len)
    end_idx = start_idx*clip_len
    indices = np.linspace(start_idx, end_idx, num=clip_len)
    indices = np.clip(indices, start_idx, end_idx - 1).astype(np.int64)
    return indices

def predict_step(images, model):
  pixel_values = feature_extractor(images=images, return_tensors="pt").pixel_values
  pixel_values = pixel_values.to(device)

  output_ids = model.generate(pixel_values, **gen_kwargs)

  preds = tokenizer_img.batch_decode(output_ids, skip_special_tokens=True)
  preds = [pred.strip() for pred in preds]
  return preds

def join_strlist_video(lista):
  entra = "In the video"
  x = []
  for i in range(0,len(lista)):
    if i !=len(lista)-1:
      entra1 = "in the frame {} is {},".format(i+1,lista[i][0])
    else:
      entra1 = "in the frame {} is {}.".format(i+1,lista[i][0])
    x.append(entra1)
  salida = " ".join(x)
  salida = "{} {}".format(entra,salida)
  return salida

In [ ]:
def youtube_commenter(url, max_len=256):
  
  # youtube to mp4
  yt = YouTube(url)
  streams = yt.streams.filter(file_extension='mp4')
  file_path = streams[0].download()
  videoreader = VideoReader(file_path, num_threads=1, ctx=cpu(0))
  print("Video descargado compadre")

  # sample 16 frames
  videoreader.seek(0)
  indices = sample_frame_indices(clip_len=16, frame_sample_rate=6, seg_len=len(videoreader))
  video = videoreader.get_batch(indices).asnumpy()
  print("Video a 16 frames compadre")

  # mp4 to mp3
  clip = mp.VideoFileClip(file_path)
  clip.audio.write_audiofile("audio.mp3")
  batch = []
  speech_array, sampling_rate = librosa.load("audio.mp3", sr=16_000)
  sampling_rate = sampling_rate
  input_values = processor(speech_array, sampling_rate=16_000, return_tensors="pt", padding=True)
  print("Video a mp3 compadre")
  
  ###############

  # predit video
  predict = list()
  for i in range(video.shape[0]):
    predict.append(predict_step(video[i],model_image)) 
  print("Video descrito compadre")

  # predit sound
  with torch.no_grad():
      logits = model_sound(input_values.input_values).logits
  predicted_ids = torch.argmax(logits, dim=-1)
  predicted_sentences = processor.batch_decode(predicted_ids)
  print("Sonido descrito compadre")

  ##############

  os.remove(file_path)
  os.remove("audio.mp3")
  print("Archivos eliminados compadre")

  ##############

  predict_video = join_strlist_video(predict)
  predict_audio = ' '.join(predicted_sentences)
  description = ''.join([predict_video] + [" The sound says "] + [predict_audio])

  text =  "headline: " + description
  print("Descripción hecha compadre")

  #############

  # generate comment  
  encoding = tokenizer_txt.encode_plus(text, return_tensors = "pt")
  input_ids = encoding["input_ids"]
  attention_masks = encoding["attention_mask"]

  beam_outputs = model.generate(
      input_ids = input_ids,
      attention_mask = attention_masks,
      max_length = max_len,
      num_beams = 3,
      early_stopping = True,
  )

  print("Comentario listo compadre")

  result = tokenizer_txt.decode(beam_outputs[0])
  
  return result, text


In [ ]:
url = "https://youtu.be/OPNZsKgzHGw"
comment, text = youtube_commenter(url,max_len=64)

Video descargado compadre
Video a 16 frames compadre
[MoviePy] Writing audio in audio.mp3


100%|██████████| 169/169 [00:00<00:00, 641.57it/s]

[MoviePy] Done.



  warnings.warn("PySoundFile failed. Trying audioread instead.")



Video a mp3 compadre
Video descrito compadre
Sonido descrito compadre
Archivos eliminados compadre
Descripción hecha compadre
Comentario listo compadre


In [ ]:
comment

'<pad> The Sound of a Dog</s>'

In [ ]:
text

"in the video in the frame 1 is a dog is sitting in the back of a pickup truck, in the frame 2 is a man is standing in the back of a pickup truck, in the frame 3 is a police car with a sign on the side of it, in the frame 4 is a dog sitting in the back seat of a car, in the frame 5 is a dog sitting in the driver's seat of a truck, in the frame 6 is a dog sitting in the back of a pickup truck, in the frame 7 is a car parked in front of a parking meter, in the frame 8 is cars are driving down the road, in the frame 9 is a car parked in a parking lot next to a street, in the frame 10 is the reflection of a car in a rear view mirror, in the frame 11 is a cat is looking out the window of a car, in the frame 12 is a dog looking out the window of a car, in the frame 13 is a dog sitting in the driver's seat of a truck, in the frame 14 is a dog sitting in the driver's seat of a car, in the frame 15 is a dog looking out the window of a car, in the frame 16 is a black and white dog looking at the